In [1]:
import sqlite3
import pandas as pd

In [2]:
path = r"C:\Users\mches\MYPROJECT\mywebapp\amazon_reviews.db"
conn = sqlite3.connect(path)
cur = conn.cursor()

In [3]:
df = pd.read_sql_query("SELECT product_title, review_body, star_rating from books LIMIT 900;", conn)
df

,product_title,review_body,star_rating
0,There Was an Old Lady Who Swallowed a Shell!,I love it and so does my students!,5
1,I Saw a Friend,My wife and I ordered 2 books and gave them as...,5
2,"Black Lagoon, Vol. 6",Great book just like all the others in the ser...,5
3,If I Stay,So beautiful,5
4,Stars 'N Strips Forever,Enjoyed the author's story and his quilts are ...,5
5,The Liar,Two or three pages into the book I suspected h...,2
6,Devil in the Details (Book 2: The Monastery Mu...,"&#34;Secrets in the Shallows,&#34; Book 1 of &...",5
7,Knowing When to Stop: A Memoir,I love it!,5
8,The American Pageant,It was a great purchase.,5
9,Punjabi C.L. Bible / Common Language Version,Quality product fast shipping.,5


In [4]:
from textblob import TextBlob
from numpy import interp

In [5]:
def sentiment(row):
    #raw text of review
    blob = TextBlob(row['review_body'])
    #sentiment polarity score
    #global polarity
    polarity = blob.sentiment.polarity
    score = interp(polarity, [-1,1], [1,5])
    return score

In [7]:
#df['sentiment_polarity'] = df.apply(sentiment,axis=1)
df['adjusted_star_rating'] = df.apply(sentiment,axis=1)
df['star_rating']=df['star_rating'].astype(float)
df['difference'] = abs(df['star_rating'] - df['adjusted_star_rating'])
#df['difference'] = (df['star_rating'].mean()) - (df['adjusted_star_rating'].mean())
df

,product_title,review_body,star_rating,adjusted_star_rating,difference
0,There Was an Old Lady Who Swallowed a Shell!,I love it and so does my students!,5.0,4.250000,0.750000
1,I Saw a Friend,My wife and I ordered 2 books and gave them as...,5.0,3.425016,1.574984
2,"Black Lagoon, Vol. 6",Great book just like all the others in the ser...,5.0,4.600000,0.400000
3,If I Stay,So beautiful,5.0,4.700000,0.300000
4,Stars 'N Strips Forever,Enjoyed the author's story and his quilts are ...,5.0,4.400000,0.600000
5,The Liar,Two or three pages into the book I suspected h...,2.0,3.336086,1.336086
6,Devil in the Details (Book 2: The Monastery Mu...,"&#34;Secrets in the Shallows,&#34; Book 1 of &...",5.0,3.483333,1.516667
7,Knowing When to Stop: A Memoir,I love it!,5.0,4.250000,0.750000
8,The American Pageant,It was a great purchase.,5.0,4.600000,0.400000
9,Punjabi C.L. Bible / Common Language Version,Quality product fast shipping.,5.0,3.400000,1.600000


In [8]:
df['difference'].mean()

1.0515998642575701

In [9]:
cur.close()
conn.close()